In [1]:
import pandas as pd
import numpy as np
from sklearn.svm import SVR
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error
import joblib
import optuna  # For hyperparameter tuning

# Tickers
tickers = ["BNB-USD", "BTC-USD", "ETH-USD", "XRP-USD"]

# Prepare data with feature engineering
def prepare_data(df):
    features = ["Open", "High", "Low", "Close", "Volume", "Lag1", "Lag7", "SMA7", "RSI14", "MACD", "Returns", "Volatility"]
    X = df[features].values
    y = df["Target"].values
    return X, y, features

# Objective function for Optuna hyperparameter tuning
def objective(trial, X_train_scaled, y_train, X_test_scaled, y_test):
    params = {
        'kernel': trial.suggest_categorical('kernel', ['rbf', 'poly']),
        'C': trial.suggest_float('C', 1e-1, 1e3, log=True),
        'epsilon': trial.suggest_float('epsilon', 0.01, 0.2, log=True),
        'gamma': trial.suggest_float('gamma', 1e-4, 1e-1, log=True) if trial.params['kernel'] == 'rbf' else None
    }
    
    if params['kernel'] == 'poly':
        params['degree'] = trial.suggest_int('degree', 2, 5)
    
    model = SVR(**{k: v for k, v in params.items() if v is not None})
    model.fit(X_train_scaled, y_train)
    y_pred = model.predict(X_test_scaled)
    mse = mean_squared_error(y_test, y_pred)
    return mse

# Train and evaluate SVR model
for ticker in tickers:
    print(f"\nTraining SVR for {ticker}...")
    
    # Load data
    df = pd.read_csv(f"{ticker}_processed.csv", index_col="Date", parse_dates=True)
    X, y, feature_names = prepare_data(df)
    
    # Split data (80% train, 20% test, no shuffle for time series)
    train_size = int(len(X) * 0.8)
    X_train, X_test = X[:train_size], X[train_size:]
    y_train, y_test = y[:train_size], y[train_size:]
    
    # Scale features
    scaler_X = StandardScaler()
    X_train_scaled = scaler_X.fit_transform(X_train)
    X_test_scaled = scaler_X.transform(X_test)
    
    # Hyperparameter tuning with Optuna
    study = optuna.create_study(direction='minimize')
    study.optimize(lambda trial: objective(trial, X_train_scaled, y_train, X_test_scaled, y_test), n_trials=30)
    best_params = study.best_params
    print(f"Best parameters for {ticker}: {best_params}")
    
    # Train final model with best parameters
    model = SVR(**best_params)
    model.fit(X_train_scaled, y_train)
    
    # Predict and evaluate
    y_pred = model.predict(X_test_scaled)
    mse = mean_squared_error(y_test, y_pred)
    mape = mean_absolute_percentage_error(y_test, y_pred) * 100
    test_dates = df.index[train_size:]
    actual_series = pd.Series(y_test, index=test_dates)
    pred_series = pd.Series(y_pred, index=test_dates)
    actual_direction = (actual_series.shift(-1) > actual_series).iloc[:-1].astype(int)
    predicted_direction = (pred_series.shift(-1) > pred_series).iloc[:-1].astype(int)
    directional_accuracy = (actual_direction == predicted_direction).mean() * 100
    
    print(f"SVR Results for {ticker}:")
    print(f"MSE: {mse:.2f}")
    print(f"MAPE: {mape:.2f}%")
    print(f"Directional Accuracy: {directional_accuracy:.2f}%")
    
    # Save model and scaler
    joblib.dump(model, f"{ticker}_svr_model.pkl")
    joblib.dump(scaler_X, f"{ticker}_scaler_X_svr.pkl")
    print(f"Model and scaler saved for {ticker}")

[I 2025-04-01 23:45:34,382] A new study created in memory with name: no-name-3b5ce991-6168-48f5-ba72-0fe078f62232
[I 2025-04-01 23:45:34,414] Trial 0 finished with value: 1160.474103167472 and parameters: {'kernel': 'rbf', 'C': 298.40667034202227, 'epsilon': 0.015551145438463632, 'gamma': 0.021183265119696634}. Best is trial 0 with value: 1160.474103167472.
[I 2025-04-01 23:45:34,434] Trial 1 finished with value: 460.2208722794747 and parameters: {'kernel': 'rbf', 'C': 347.62781417833946, 'epsilon': 0.03295351034802889, 'gamma': 0.00018400848624248832}. Best is trial 1 with value: 460.2208722794747.
[I 2025-04-01 23:45:34,454] Trial 2 finished with value: 428.7181221474715 and parameters: {'kernel': 'rbf', 'C': 560.7878268738478, 'epsilon': 0.015531041167123973, 'gamma': 0.00014058524589011503}. Best is trial 2 with value: 428.7181221474715.
[I 2025-04-01 23:45:34,471] Trial 3 finished with value: 98662.79271499475 and parameters: {'kernel': 'poly', 'C': 2.2793742046818743, 'epsilon': 


Training SVR for BNB-USD...


[I 2025-04-01 23:45:34,559] Trial 7 finished with value: 33213.20483894761 and parameters: {'kernel': 'poly', 'C': 1.462396725130801, 'epsilon': 0.07594541910947479, 'degree': 2}. Best is trial 2 with value: 428.7181221474715.
[I 2025-04-01 23:45:34,584] Trial 8 finished with value: 19207.688032640628 and parameters: {'kernel': 'poly', 'C': 86.90893015110494, 'epsilon': 0.04044449364855417, 'degree': 2}. Best is trial 2 with value: 428.7181221474715.
[I 2025-04-01 23:45:34,610] Trial 9 finished with value: 677.7088610173886 and parameters: {'kernel': 'rbf', 'C': 39.65721493969196, 'epsilon': 0.021880807490721722, 'gamma': 0.0035976258897861325}. Best is trial 2 with value: 428.7181221474715.
[I 2025-04-01 23:45:34,642] Trial 10 finished with value: 661.8932544511275 and parameters: {'kernel': 'rbf', 'C': 360.3742300504328, 'epsilon': 0.010067155261093263, 'gamma': 0.0001015993642000894}. Best is trial 2 with value: 428.7181221474715.
[I 2025-04-01 23:45:34,673] Trial 11 finished with v

Best parameters for BNB-USD: {'kernel': 'rbf', 'C': 979.1564200924081, 'epsilon': 0.0679144640392676, 'gamma': 0.000855347967566496}
SVR Results for BNB-USD:
MSE: 363.48
MAPE: 2.13%
Directional Accuracy: 46.43%
Model and scaler saved for BNB-USD

Training SVR for BTC-USD...


[I 2025-04-01 23:45:35,774] Trial 8 finished with value: 207261536536.8688 and parameters: {'kernel': 'poly', 'C': 14.129845360032805, 'epsilon': 0.031199697557435627, 'degree': 5}. Best is trial 3 with value: 1826029904.3451936.
[I 2025-04-01 23:45:35,787] Trial 9 finished with value: 2512276312.105695 and parameters: {'kernel': 'rbf', 'C': 0.3588109075295044, 'epsilon': 0.14812494507555157, 'gamma': 0.024174312305775977}. Best is trial 3 with value: 1826029904.3451936.
[I 2025-04-01 23:45:35,819] Trial 10 finished with value: 2460213120.2328734 and parameters: {'kernel': 'poly', 'C': 3.098336245827362, 'epsilon': 0.010990614450636158, 'degree': 2}. Best is trial 3 with value: 1826029904.3451936.
[I 2025-04-01 23:45:35,838] Trial 11 finished with value: 1602421365.0729082 and parameters: {'kernel': 'poly', 'C': 2.262285943256586, 'epsilon': 0.061624021473382126, 'degree': 4}. Best is trial 11 with value: 1602421365.0729082.
[I 2025-04-01 23:45:35,867] Trial 12 finished with value: 114

Best parameters for BTC-USD: {'kernel': 'poly', 'C': 5.115290200179372, 'epsilon': 0.11454968320349496, 'degree': 3}
SVR Results for BTC-USD:
MSE: 575552081.57
MAPE: 24.56%
Directional Accuracy: 46.43%
Model and scaler saved for BTC-USD

Training SVR for ETH-USD...


[I 2025-04-01 23:45:36,509] Trial 9 finished with value: 4041537.1592668183 and parameters: {'kernel': 'poly', 'C': 7.216443964491101, 'epsilon': 0.05636621502020363, 'degree': 5}. Best is trial 3 with value: 240295.2565824215.
[I 2025-04-01 23:45:36,540] Trial 10 finished with value: 39975.59262781184 and parameters: {'kernel': 'rbf', 'C': 924.1431346302288, 'epsilon': 0.1650951318456754, 'gamma': 0.00010039649888672384}. Best is trial 10 with value: 39975.59262781184.
[I 2025-04-01 23:45:36,567] Trial 11 finished with value: 40253.05867757652 and parameters: {'kernel': 'rbf', 'C': 762.5872223122758, 'epsilon': 0.17850669765497837, 'gamma': 0.00012088301549379822}. Best is trial 10 with value: 39975.59262781184.
[I 2025-04-01 23:45:36,592] Trial 12 finished with value: 38135.31711939982 and parameters: {'kernel': 'rbf', 'C': 955.2890763764342, 'epsilon': 0.18520861425664137, 'gamma': 0.00010040348657711895}. Best is trial 12 with value: 38135.31711939982.
[I 2025-04-01 23:45:36,622] T

Best parameters for ETH-USD: {'kernel': 'rbf', 'C': 201.20367421673234, 'epsilon': 0.033591036715239164, 'gamma': 0.0018137414003431705}
SVR Results for ETH-USD:
MSE: 16891.67
MAPE: 3.38%
Directional Accuracy: 47.14%
Model and scaler saved for ETH-USD

Training SVR for XRP-USD...


[I 2025-04-01 23:45:37,385] Trial 1 finished with value: 1300346728810.2673 and parameters: {'kernel': 'poly', 'C': 863.5960138896837, 'epsilon': 0.1447024086889199, 'degree': 5}. Best is trial 0 with value: 49206147088.23212.
[I 2025-04-01 23:45:37,390] Trial 2 finished with value: 2.6574257539219746 and parameters: {'kernel': 'rbf', 'C': 3.8128944455637965, 'epsilon': 0.06720278645715343, 'gamma': 0.0038301561292719256}. Best is trial 2 with value: 2.6574257539219746.
[I 2025-04-01 23:45:37,408] Trial 3 finished with value: 2.41870509065046 and parameters: {'kernel': 'rbf', 'C': 13.79763959211851, 'epsilon': 0.014089689415647578, 'gamma': 0.006525481297250016}. Best is trial 3 with value: 2.41870509065046.
[I 2025-04-01 23:45:37,885] Trial 4 finished with value: 617739.8773050692 and parameters: {'kernel': 'poly', 'C': 18.43583335998929, 'epsilon': 0.03412844602960292, 'degree': 3}. Best is trial 3 with value: 2.41870509065046.
[I 2025-04-01 23:45:38,484] Trial 5 finished with value:

Best parameters for XRP-USD: {'kernel': 'rbf', 'C': 115.37146300941903, 'epsilon': 0.0398140261832481, 'gamma': 0.00010012373474877815}
SVR Results for XRP-USD:
MSE: 0.38
MAPE: 19.15%
Directional Accuracy: 50.71%
Model and scaler saved for XRP-USD
